In [9]:
import numpy as np
import math
from models import interpolation, SRCNN_train, SRCNN_model, SRCNN_predict, DNCNN_train, DNCNN_model, DNCNN_predict
#from scipy.misc import imresize
from scipy.io import loadmat
import matplotlib.pyplot as plt

In [10]:
import os
os.path.isfile("C:/Users/golam/Documents/ChannelNet-master/Perfect_H_40000.mat")  #这里最好设置一个指定的文件路径，不要设置相对文件路径
os.path.isfile("C:/Users/golam/Documents/ChannelNet-master/My_noisy_H_22.mat")   #这里最好设置一个指定的文件路径，不要设置相对文件路径


True

In [11]:
# data:Perfect_H_40000.mat
# noisy: My_noisy_H_22.mat
if __name__ == "__main__":
    # load datasets 
    channel_model = "VehA"
    SNR = 22
    Number_of_pilots = 48
    perfect = loadmat("C:/Users/golam/Documents/ChannelNet-master/Perfect_H_40000.mat")['My_perfect_H']
    noisy_input = loadmat("C:/Users/golam/Documents/ChannelNet-master/My_noisy_H_22.mat")['My_noisy_H']

In [12]:
interp_noisy = interpolation(noisy_input , SNR , Number_of_pilots , 'rbf')


start!


100%|███████████████████████████████████████████████████████████████████████████| 40000/40000 [02:50<00:00, 234.42it/s]


In [13]:
#     interp_noisy = np.load("interp_noisy.npy")
#     np.save('interp_noisy.npy',interp_noisy)

In [14]:
perfect_image = np.zeros((len(perfect),72,14,2))
perfect_image[:,:,:,0] = np.real(perfect)
perfect_image[:,:,:,1] = np.imag(perfect)
perfect_image = np.concatenate((perfect_image[:,:,:,0], perfect_image[:,:,:,1]), axis=0).reshape(2*len(perfect), 72, 14, 1)

In [15]:
####### ------ training SRCNN ------ #######

idx_random = np.random.rand(len(perfect_image)) < (1/9)  # uses 32000 from 36000 as training and the rest as validation
train_data, train_label = interp_noisy[idx_random,:,:,:] , perfect_image[idx_random,:,:,:]
val_data, val_label = interp_noisy[~idx_random,:,:,:] , perfect_image[~idx_random,:,:,:]

In [16]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [17]:
SRCNN_model()

In [18]:
SRCNN_train(train_data ,train_label, val_data , val_label , channel_model , Number_of_pilots , SNR )

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 72, 14, 1)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 72, 14, 64)        5248      
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 14, 32)        2080      
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 14, 1)         801       
                                                                 
Total params: 8129 (31.75 KB)
Trainable params: 8129 (31.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None

Epoch 1: val_loss improved from inf to 0.00964, saving model to SRCNN_check.h5


C:\Users\golam\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 0.00964 to 0.00499, saving model to SRCNN_check.h5

Epoch 3: val_loss improved from 0.00499 to 0.00331, saving model to SRCNN_check.h5

Epoch 4: val_loss improved from 0.00331 to 0.00275, saving model to SRCNN_check.h5

Epoch 5: val_loss improved from 0.00275 to 0.00247, saving model to SRCNN_check.h5

Epoch 6: val_loss improved from 0.00247 to 0.00228, saving model to SRCNN_check.h5

Epoch 7: val_loss did not improve from 0.00228

Epoch 8: val_loss improved from 0.00228 to 0.00207, saving model to SRCNN_check.h5

Epoch 9: val_loss improved from 0.00207 to 0.00199, saving model to SRCNN_check.h5

Epoch 10: val_loss improved from 0.00199 to 0.00198, saving model to SRCNN_check.h5

Epoch 11: val_loss improved from 0.00198 to 0.00196, saving model to SRCNN_check.h5

Epoch 12: val_loss improved from 0.00196 to 0.00193, saving model to SRCNN_check.h5

Epoch 13: val_loss did not improve from 0.00193

Epoch 14: val_loss improved from 0.00193 to 0.00191, saving


Epoch 146: val_loss did not improve from 0.00137

Epoch 147: val_loss did not improve from 0.00137

Epoch 148: val_loss did not improve from 0.00137

Epoch 149: val_loss did not improve from 0.00137

Epoch 150: val_loss did not improve from 0.00137

Epoch 151: val_loss did not improve from 0.00137

Epoch 152: val_loss did not improve from 0.00137

Epoch 153: val_loss did not improve from 0.00137

Epoch 154: val_loss did not improve from 0.00137

Epoch 155: val_loss improved from 0.00137 to 0.00137, saving model to SRCNN_check.h5

Epoch 156: val_loss did not improve from 0.00137

Epoch 157: val_loss did not improve from 0.00137

Epoch 158: val_loss did not improve from 0.00137

Epoch 159: val_loss did not improve from 0.00137

Epoch 160: val_loss did not improve from 0.00137

Epoch 161: val_loss did not improve from 0.00137

Epoch 162: val_loss did not improve from 0.00137

Epoch 163: val_loss did not improve from 0.00137

Epoch 164: val_loss did not improve from 0.00137

Epoch 165: va

After training 300 epochs, the val_loss improved from inf to 0.00133, saving model to SRCNN_VehA_48_22.h5.

In [19]:
####### ------ prediction using SRCNN ------ #######
srcnn_pred_train = SRCNN_predict(train_data, channel_model , Number_of_pilots , SNR) 

277/277 [==============================] - 11s 28ms/step


In [20]:
srcnn_pred_validation = SRCNN_predict(val_data, channel_model , Number_of_pilots , SNR)

2224/2224 [==============================] - 63s 28ms/step


In [21]:
print(srcnn_pred_train.shape)

(8853, 72, 14, 1)


In [22]:
# save the result of prediction
np.savetxt('srcnn_pred_train.txt',srcnn_pred_train[0][0])
np.savetxt('srcnn_pred_validation.txt',srcnn_pred_validation[0][0])

In [27]:
DNCNN_model()

In [28]:
####### ------ training DNCNN ------ #######
DNCNN_train(train_data ,train_label, val_data , val_label , channel_model , Number_of_pilots , SNR )

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, None, None, 1)]      0         []                            
                                                                                                  
 conv2d_112 (Conv2D)         (None, None, None, 64)       640       ['input_10[0][0]']            
                                                                                                  
 activation_95 (Activation)  (None, None, None, 64)       0         ['conv2d_112[0][0]']          
                                                                                                  
 conv2d_113 (Conv2D)         (None, None, None, 64)       36928     ['activation_95[0][0]']       
                                                                                            

 batch_normalization_99 (Ba  (None, None, None, 64)       256       ['conv2d_122[0][0]']          
 tchNormalization)                                                                                
                                                                                                  
 activation_105 (Activation  (None, None, None, 64)       0         ['batch_normalization_99[0][0]
 )                                                                  ']                            
                                                                                                  
 conv2d_123 (Conv2D)         (None, None, None, 64)       36928     ['activation_105[0][0]']      
                                                                                                  
 batch_normalization_100 (B  (None, None, None, 64)       256       ['conv2d_123[0][0]']          
 atchNormalization)                                                                               
          

KeyboardInterrupt: 

After training 200 epochs, the val_loss improved from 0.0935 to 0.00106, saving model to DNCNN_VehA_48_22.h5.

In [ ]:
`